# Интеллектуальный анализ данных – весна 2025
# Домашнее задание 6: классификация текстов

Правила:



*   Домашнее задание оценивается в 10 баллов.
*   Можно использовать без доказательства любые результаты, встречавшиеся на лекциях или семинарах по курсу, если получение этих результатов не является вопросом задания.
*  Можно использовать любые свободные источники с *обязательным* указанием ссылки на них.
*  Плагиат не допускается. При обнаружении случаев списывания, 0 за работу выставляется всем участникам нарушения, даже если можно установить, кто у кого списал.
*  Старайтесь сделать код как можно более оптимальным. В частности, будет штрафоваться использование циклов в тех случаях, когда операцию можно совершить при помощи инструментов библиотек, о которых рассказывалось в курсе.
* Если в задании есть вопрос на рассуждение, то за отсутствие ответа на него балл за задание будет снижен вполовину.

В этом домашнем задании вам предстоит построить классификатор текстов.

Будем предсказывать эмоциональную окраску твиттов о коронавирусе.



In [32]:
import numpy as np
import pandas as pd
from typing import  List
import matplotlib.pyplot as plt
import seaborn as sns
from string import punctuation

In [33]:
df = pd.read_csv('/content/tweets_coronavirus.csv', encoding='latin-1')
df.sample(4)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
24496,33767,78719,"Miami, FL",06-04-2020,Last month when governors stepped up to protec...,Positive
4861,9699,54651,Solihull,19-03-2020,Birmingham mums misery at FOUR WEEK wait for s...,Negative
20360,28636,73588,"Los Angeles, CA",27-03-2020,#Work #Labor #Pandemic #Coronavirus #COVID?19 ...,Extremely Positive
11155,17335,62287,"Adelaide, South Australia",21-03-2020,"@Vic_Rollison ""Have some dignity."" Our Secreta...",Positive


Для каждого твитта указано:


*   UserName - имя пользователя, заменено на целое число для анонимности
*   ScreenName - отображающееся имя пользователя, заменено на целое число для анонимности
*   Location - местоположение
*   TweetAt - дата создания твитта
*   OriginalTweet - текст твитта
*   Sentiment - эмоциональная окраска твитта (целевая переменная)



## Задание 1 Подготовка (0.5 балла)

Целевая переменная находится в колонке `Sentiment`.  Преобразуйте ее таким образом, чтобы она стала бинарной: 1 - если у твитта положительная или очень положительная эмоциональная окраска и 0 - если отрицательная или очень отрицательная.

In [34]:
# -- YOUR CODE HERE --
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x in ['Positive', 'Extremely Positive'] else 0)
df

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,1
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,1
2,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,1
3,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0
4,3804,48756,"ÃÂT: 36.319708,-82.363649",16-03-2020,As news of the regionÃÂs first confirmed COV...,1
...,...,...,...,...,...,...
33439,44949,89901,OHIO,14-04-2020,I never that weÃÂd be in a situation &amp; w...,1
33440,44950,89902,NaN,14-04-2020,@MrSilverScott you are definitely my man. I fe...,1
33441,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,0
33442,44953,89905,NaN,14-04-2020,You know itÃÂs getting tough when @KameronWi...,1


Сбалансированы ли классы?

In [35]:
# -- YOUR CODE HERE --
total = df['Sentiment'].value_counts().sum()
part_1 = ((df['Sentiment'] == 1).sum())/total * 100
part_0 = ((df['Sentiment'] == 0).sum())/total * 100
print(f'баланс классов: {part_1:.2f}% / {part_0:.2f}%')

баланс классов: 53.96% / 46.04%


**Ответ:** # -- YOUR ANSWER HERE --
 **Да. Классы почти идеально сбалансированы**

Выведете на экран информацию о пропусках в данных. Если пропуски присутствуют заполните их строкой 'Unknown'.

In [36]:
# -- YOUR CODE HERE --
print('Информация о пропусках в данных')
print(df.isna().sum())
df = df.fillna('Unknown')

Информация о пропусках в данных
UserName            0
ScreenName          0
Location         7049
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64


Разделите данные на обучающие и тестовые в соотношении 7 : 3 и укажите `random_state=0`

In [37]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, random_state=0) # -- YOUR CODE HERE --

## Задание 2 Токенизация (3 балла)

Постройте словарь на основе обучающей выборки и посчитайте количество встреч каждого токена с использованием самой простой токенизации - деления текстов по пробельным символам и приведения токенов в нижний регистр.

In [38]:
# -- YOUR CODE HERE --
tokens_dict = {}
for tweet in df['OriginalTweet']:
  tokenized_tweet = str(tweet).lower().split()
  for token in tokenized_tweet:
    if token in tokens_dict:
      tokens_dict[token] += 1
    else:
      tokens_dict[token] = 1
tokens_dict

{'advice': 195,
 'talk': 115,
 'to': 33447,
 'your': 3999,
 'neighbours': 24,
 'family': 392,
 'exchange': 25,
 'phone': 114,
 'numbers': 72,
 'create': 117,
 'contact': 271,
 'list': 250,
 'with': 5764,
 'of': 18578,
 'schools': 78,
 'employer': 17,
 'chemist': 19,
 'gp': 9,
 'set': 231,
 'up': 2875,
 'online': 2388,
 'shopping': 2300,
 'accounts': 34,
 'if': 3058,
 'poss': 1,
 'adequate': 30,
 'supplies': 537,
 'regular': 101,
 'meds': 18,
 'but': 3082,
 'not': 3873,
 'over': 1112,
 'order': 485,
 'coronavirus': 1376,
 'australia:': 1,
 'woolworths': 15,
 'give': 404,
 'elderly,': 19,
 'disabled': 69,
 'dedicated': 63,
 'hours': 389,
 'amid': 798,
 'covid-19': 4504,
 'outbreak': 573,
 'https://t.co/binca9vp8p': 1,
 'my': 3557,
 'food': 5409,
 'stock': 1525,
 'is': 10596,
 'the': 38250,
 'only': 1188,
 'one': 1548,
 'which': 729,
 'empty...': 3,
 'please,': 36,
 "don't": 778,
 'panic,': 71,
 'there': 1710,
 'will': 3865,
 'be': 5058,
 'enough': 501,
 'for': 12193,
 'everyone': 854,
 '

Какой размер словаря получился?

In [39]:
# -- YOUR CODE HERE --
len(tokens_dict)

103200

Выведите 10 самых популярных токенов с количеством встреч каждого из них. Объясните, почему именно эти токены в топе.

In [40]:
# -- YOUR CODE HERE --
for token, count in (sorted(tokens_dict.items(), key=lambda item: item[1], reverse=True)[:10]):
  print(f'{token}: {count}')

the: 38250
to: 33447
and: 20935
of: 18578
a: 16667
in: 16024
for: 12193
#coronavirus: 11759
is: 10596
are: 9958


**Ответ:** # -- YOUR ANSWER HERE --
**Эти токены в топе из-за того, что эти слова - служебные для английского языка (частицы, предлоги и т.д.). Еще есть слово "коронавирус", которое здесь по причине того, что датафрейм о твитах про коронавирус.**

Удалите стоп-слова из словаря и выведите новый топ-10 токенов (и количество встреч) по популярности.  Что можно сказать  о нем?

In [41]:
!pip install nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
import nltk
from nltk.corpus import stopwords

# -- YOUR CODE HERE --
tokens_dict_without_stopwords = dict(filter(lambda item: item[0] not in stopwords.words("english"), tokens_dict.items()))
for token, count in (sorted(tokens_dict_without_stopwords.items(), key=lambda item: item[1], reverse=True)[:10]):
  print(f'{token}: {count}')

#coronavirus: 11759
prices: 5625
food: 5409
grocery: 4882
supermarket: 4662
covid-19: 4504
people: 4488
store: 4486
#covid19: 3561
consumer: 3233


**Ответ:**  # -- YOUR ANSWER HERE --
**Про список можно сказать, что 3 слова - коронавирус. Есть слово люди. Остальные слова связаны с магазинами. Выглядит правдободобно.**

Также выведите 20 самых непопулярных слов (если самых непопулярных слов больше, выведите любые 20 из них) Почему эти токены непопулярны, требуется ли как-то дополнительно работать с ними?

In [44]:
# -- YOUR CODE HERE --
for token, count in (sorted(tokens_dict_without_stopwords.items(), key=lambda item: item[1])[:20]):
  print(f'{token}: {count}')

poss: 1
australia:: 1
https://t.co/binca9vp8p: 1
#confinementgeneral: 1
https://t.co/zrlg0z520j: 1
litteraly: 1
#coronavirusfrance: 1
https://t.co/usmualq72n: 1
@tim_dodson: 1
https://t.co/cfxch7a2lu: 1
civics: 1
about".: 1
https://t.co/iefdnehgdo: 1
https://t.co/kw91zj5o5i: 1
prevention,we: 1
19?.: 1
#govindia: 1
#horningsea: 1
https://t.co/lsgrxxhjhh: 1
https://t.co/8ywakfjexc: 1


**Ответ:** # -- YOUR ANSWER HERE --
Эти токены непопулярны, потому что они уникальны. Например: ссылки, упоминания людей через @, непопулярные хэштэги, ошибка в слове "litteraly". Что-то стало уникальным из-за токенизации, например: 19?.
Дополнительно с ними работать не требуется, так как они имеют очень маленькую значимость и не влияют на общую картину.


Теперь воспользуемся токенайзером получше - TweetTokenizer из библиотеки nltk. Примените его и посмотрите на топ-10 популярных слов. Чем он отличается от топа, который получался раньше? Почему?

In [47]:
from nltk.tokenize import TweetTokenizer

# -- YOUR CODE HERE --
tw = TweetTokenizer()
tweet_tokenized_dict = {}
for tweet in df['OriginalTweet']:
  tw_tokenized_tweet = tw.tokenize(tweet)
  for token in tw_tokenized_tweet:
    if token in tweet_tokenized_dict:
      tweet_tokenized_dict[token] += 1
    else:
      tweet_tokenized_dict[token] = 1
tweet_tokenized_dict


{'advice': 186,
 'Talk': 12,
 'to': 32812,
 'your': 3766,
 'neighbours': 33,
 'family': 442,
 'exchange': 20,
 'phone': 124,
 'numbers': 88,
 'create': 111,
 'contact': 246,
 'list': 277,
 'with': 5332,
 'of': 18429,
 'schools': 76,
 'employer': 18,
 'chemist': 18,
 'GP': 12,
 'set': 219,
 'up': 3053,
 'online': 2376,
 'shopping': 2490,
 'accounts': 37,
 'if': 1762,
 'poss': 1,
 'adequate': 26,
 'supplies': 683,
 'regular': 92,
 'meds': 28,
 'but': 2577,
 'not': 3443,
 'over': 1172,
 'order': 475,
 'Coronavirus': 623,
 'Australia': 107,
 ':': 4944,
 'Woolworths': 16,
 'give': 347,
 'elderly': 376,
 ',': 25142,
 'disabled': 68,
 'dedicated': 59,
 'hours': 435,
 'amid': 710,
 'COVID': 5945,
 '-': 9446,
 '19': 10142,
 'outbreak': 855,
 'https://t.co/bInCA9Vp8P': 1,
 'My': 723,
 'food': 5423,
 'stock': 1521,
 'is': 10246,
 'the': 34781,
 'only': 1093,
 'one': 1380,
 'which': 703,
 'empty': 592,
 '...': 2458,
 'PLEASE': 108,
 "don't": 621,
 'panic': 2068,
 'THERE': 18,
 'WILL': 45,
 'BE': 4

**Ответ:** # -- YOUR ANSWER HERE --

Удалите из словаря стоп-слова и пунктуацию, посмотрите на новый топ-10 слов с количеством встреч, есть ли теперь в нем что-то не похожее на слова?

In [ ]:
from string import punctuation

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Скорее всего в некоторых топах были неотображаемые символы или отдельные буквы не латинского алфавита. Уберем их: удалите из словаря токены из одного символа, позиция которого в таблице Unicode 128 и более (`ord(x) >= 128`)

Выведите топ-10 самых популярных и топ-20 непопулярных слов. Чем полученные топы отличаются от итоговых топов, полученных при использовании токенизации по пробелам? Что теперь лучше, а что хуже?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Выведите топ-10 популярных хештегов (токены, первые символы которых - #) с количеством встреч. Что можно сказать о них?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

То же самое проделайте для ссылок на сайт https://t.co Сравнима ли популярность ссылок с популярностью хештегов? Будет ли информация о ссылке на конкретную страницу полезна?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Используем опыт предыдущих экспериментов и напишем собственный токенайзер, улучшив TweetTokenizer. Функция tokenize должна:



*   Привести текст в нижний регистр
*   Применить TweetTokenizer для  выделения токенов
*   Удалить стоп-слова, пунктуацию, токены из одного символа с позицией в таблице Unicode 128 и более,  ссылки на t.co



In [ ]:
def custom_tokenizer(text):

  # -- YOUR CODE HERE --

  return tokens


In [ ]:
custom_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext')

['sample', 'text', '@sample_text', '#sampletext']

## Задание 3 Векторизация текстов (2 балла)

Обучите CountVectorizer с использованием custom_tokenizer в качестве токенайзера. Как размер полученного словаря соотносится с размером изначального словаря из начала задания 2?

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer # -- YOUR CODE HERE --

print(len(cv.vocabulary_))

45308


**Ответ:** # -- YOUR ANSWER HERE --

Посмотрим на какой-нибудь конкретный твитт:

In [ ]:
ind = 9023
train.iloc[ind]['OriginalTweet'], train.iloc[ind]['Sentiment']

Автор твитта не доволен ситуацией с едой во Франции и текст имеет резко негативную окраску.

Примените обученный CountVectorizer для векторизации данного текста, и попытайтесь определить самый важный токен и самый неважный токен (токен, компонента которого в векторе максимальна/минимальна, без учета 0). Хорошо ли они определились, почему?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Теперь примените TfidfVectorizer и  определите самый важный/неважный токены. Хорошо ли определились, почему?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Найдите какой-нибудь положительно окрашенный твитт, где TfidfVectorizer хорошо (полезно для определения окраски) выделяет важный токен, поясните пример.

*Подсказка:* явно положительные твитты можно искать при помощи положительных слов (good, great, amazing и т. д.)

In [ ]:
train[train['OriginalTweet'].apply(lambda x: 'your_good_word_here' in x) & (train['Sentiment'] == 1)]

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 4 Обучение первых моделей (1 балл)

Примените оба векторайзера для получения матриц с признаками текстов.  Выделите целевую переменную.

In [ ]:
# -- YOUR CODE HERE --

Обучите логистическую регрессию на векторах из обоих векторайзеров. Посчитайте долю правильных ответов на обучающих и тестовых данных. Какой векторайзер показал лучший результат? Что можно сказать о моделях?

Используйте `sparse` матрицы (после векторизации), не превращайте их в `numpy.ndarray` или `pd.DataFrame` - может не хватить памяти.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 5 Стемминг (0.5 балла)

Для уменьшения словаря можно использовать стемминг.

Модифицируйте написанный токенайзер, добавив в него стемминг с использованием SnowballStemmer. Обучите Count- и Tfidf- векторайзеры. Как изменился размер словаря?

In [ ]:
def custom_stem_tokenizer(text):
  # -- YOUR CODE HERE --
  return tokens

In [ ]:
custom_stem_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext adding more words to check stemming')

['sampl', 'text', '@sample_text', '#sampletext', 'ad', 'word', 'check', 'stem']

In [ ]:
cv = CountVectorizer # -- YOUR CODE HERE --

print(len(cv.vocabulary_))

36652


**Ответ** # -- YOUR ANSWER HERE --

Обучите логистическую регрессию с использованием обоих векторайзеров. Изменилось ли качество? Есть ли смысл применять стемминг?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание  6 Работа с частотами (1.5 балла)

Еще один способ уменьшить количество признаков - это использовать параметры min_df и max_df при построении векторайзера  эти параметры помогают ограничить требуемую частоту встречаемости токена в документах.

По умолчанию берутся все токены, которые встретились хотя бы один раз.



Подберите max_df такой, что размер словаря будет 36651 (на 1 меньше, чем было). Почему параметр получился такой большой/маленький?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        max_df=# -- YOUR CODE HERE --
                        ).fit(
                            # -- YOUR CODE HERE --
                            )
print(len(cv_df.vocabulary_))

36651


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Подберите min_df (используйте дефолтное значение max_df) в CountVectorizer таким образом, чтобы размер словаря был 3700 токенов (при использовании токенайзера со стеммингом), а качество осталось таким же, как и было. Что можно сказать о результатах?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        min_df=# -- YOUR CODE HERE --
                        ).fit(
                            # -- YOUR CODE HERE --
                            )
print(len(cv_df.vocabulary_))

3700


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

В предыдущих заданиях признаки не скалировались. Отскалируйте данные (при словаре размера 3.7 тысяч, векторизованные CountVectorizer), обучите логистическую регрессию, посмотрите качество и выведите `barplot`, содержащий по 10 токенов, с наибольшим по модулю положительными/отрицательными весами. Что можно сказать об этих токенах?

In [ ]:
from sklearn.preprocessing import StandardScaler
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 7 Другие признаки (1.5 балла)

Мы были сконцентрированы на работе с текстами твиттов и не использовали другие признаки - имена пользователя, дату и местоположение

Изучите признаки UserName и ScreenName. полезны ли они? Если полезны, то закодируйте их, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Изучите признак TweetAt в обучающей выборке: преобразуйте его к типу datetime и нарисуйте его гистограмму с разделением по цвету на основе целевой переменной. Полезен ли он? Если полезен, то закодируйте его, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --



Поработайте с признаком Location в обучающей выборке. Сколько уникальных значений?

In [ ]:
# -- YOUR CODE HERE --

Постройте гистограмму топ-10 по популярности местоположений (исключая Unknown)

In [ ]:
# -- YOUR CODE HERE --

Видно, что многие местоположения включают в себя более точное название места, чем другие (Например, у некоторых стоит London, UK; а у некоторых просто UK или United Kingdom).

Создайте новый признак WiderLocation, который содержит самое широкое местоположение (например, из London, UK должно получиться UK). Сколько уникальных категорий теперь? Постройте аналогичную гистограмму.

In [ ]:
# -- YOUR CODE HERE --

Закодируйте признак WiderLocation с помощью OHE таким образом, чтобы создались только столбцы для местоположений, которые встречаются более одного раза. Сколько таких значений?


In [ ]:
# -- YOUR CODE HERE --

Добавьте этот признак к матрице отскалированных текстовых признаков, обучите логистическую регрессию, замерьте качество. Как оно изменилось? Оказался ли признак полезным?


*Подсказка:* используйте параметр `categories` в энкодере.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --